# Converting the Belgium cells from original csv file into an importable circus actor.  

In [1]:
import pandas as pd
from datagenerator.core import actor
from datagenerator.core import circus

circus = circus.Circus(name="belgium", master_seed=1234, 
                start=pd.Timestamp("22 July 2012"), 
                step_duration=pd.Timedelta("12h") )

# Distributors


### # relationship from distributor l2 to l1

In [2]:
# relationship from delear
dealer_l2_to_l1_rel = pd.read_csv("source_data/relationships/distributor_pos_product.csv")
dealer_l2_to_l1_rel = dealer_l2_to_l1_rel[dealer_l2_to_l1_rel.agent_id.str[:4] != "FIPO"]
dealer_l2_to_l1_rel = dealer_l2_to_l1_rel[dealer_l2_to_l1_rel.agent_id.str[:4] != "MOPO"]
dealer_l2_to_l1_rel = dealer_l2_to_l1_rel.rename(columns={
        "agent_id": "dist_l2",
        "distributor_id": "dist_l1"})
dealer_l2_to_l1_rel.head()


,dist_l2,dist_l1,product_type_id
4735,DIST16,DIST4,handset
4736,DIST8,DIST4,handset
4737,DIST14,DIST2,handset
4738,DIST12,DIST4,handset
4739,DIST22,DIST2,handset


###  relationship from geo_level_3 to dist l1 and dist l2

In [3]:
# first retrieving the set of geo_level_3 covereved by each distributor level 1
dist_l1_per_geo_l1 = pd.read_csv("source_data/relationships/distributor_geo_product.csv").drop("product_type_id", 
                                                                                               axis=1)

# mapping from geo level 1 to geo level 3
geo_l1_to_l3 = pd.read_csv("source_data/geography/geography.csv")[["geo_level1_id","geo_level3_id"]]
geo_l1_to_l3.geo_level3_id = geo_l1_to_l3.geo_level3_id.map(
    lambda s: "Liège" if s[:2]=="Li" and s[4:]=="ge" else s)


dist_l1_geo_l3 = pd.merge(left=dist_l1_per_geo_l1, 
                          right=geo_l1_to_l3,
                          on="geo_level1_id")\
                   .drop("geo_level1_id", axis=1)\
                   .drop_duplicates()\
                   .sort_values("distributor_id")
dist_l1_geo_l3

,distributor_id,geo_level3_id
0,DIST1,Brabant Flamand
1000,DIST1,Luxembourg
865,DIST1,Brabant Wallon
645,DIST1,Limbourg
1220,DIST1,Hainaut
325,DIST1,Flandre Occidentale
647,DIST2,Limbourg
867,DIST2,Brabant Wallon
2,DIST2,Brabant Flamand
1002,DIST2,Luxembourg


In [4]:
# second, the formatting in an identical shape the distributor l1 of each distributor 
# => this allows to assign randomly 
dist_l2_info = pd.DataFrame(
    dealer_l2_to_l1_rel.groupby("dist_l2").agg(lambda s: list(s.unique())))

dist_l2_info = dist_l2_info.reset_index()
dist_l2_info = dist_l2_info.rename(columns={"distributor_id": "dist_l1",})
dist_l2_info.dist_l1 = dist_l2_info.dist_l1.apply(lambda s: s[0])
dist_l2_info = dist_l2_info.sort_values("dist_l1")

dist_l2_info["geo_level3_id"] = dist_l1_geo_l3.geo_level3_id.values
dist_l2_info

,dist_l2,dist_l1,product_type_id,geo_level3_id
21,DIST9,DIST1,"[sim, mfs]",Brabant Flamand
17,DIST5,DIST1,"[sim, mfs]",Luxembourg
3,DIST13,DIST1,"[sim, mfs]",Brabant Wallon
15,DIST25,DIST1,"[sim, mfs]",Limbourg
7,DIST17,DIST1,"[sim, mfs]",Hainaut
11,DIST21,DIST1,"[sim, mfs]",Flandre Occidentale
18,DIST6,DIST2,"[handset, physical_recharge, electronic_recharge]",Limbourg
16,DIST26,DIST2,"[handset, physical_recharge, electronic_recharge]",Brabant Wallon
12,DIST22,DIST2,"[handset, physical_recharge, electronic_recharge]",Brabant Flamand
0,DIST10,DIST2,"[handset, physical_recharge, electronic_recharge]",Luxembourg


In [5]:
# just a reshaping to later enable the join to this table per product
def explode_row(row):
    return [[row["dist_l2"], row["dist_l1"], row["geo_level3_id"], p] 
                for p in row.product_type_id]

rows_list = dist_l2_info.apply(explode_row, axis=1)

# dealers l1 and l2 for each geo level 3
dealers_per_geo_l3 = pd.concat(
    [pd.DataFrame(rows, columns=["dist_l2", "dist_l1", "geo_level3_id", "product"]) 
       for rows in rows_list])

# this is what we want: for eath geo_level_3 and product, the corresponding dist_l1 and dist_l2
dealers_per_geo_l3

,dist_l2,dist_l1,geo_level3_id,product
0,DIST9,DIST1,Brabant Flamand,sim
1,DIST9,DIST1,Brabant Flamand,mfs
0,DIST5,DIST1,Luxembourg,sim
1,DIST5,DIST1,Luxembourg,mfs
0,DIST13,DIST1,Brabant Wallon,sim
1,DIST13,DIST1,Brabant Wallon,mfs
0,DIST25,DIST1,Limbourg,sim
1,DIST25,DIST1,Limbourg,mfs
0,DIST17,DIST1,Hainaut,sim
1,DIST17,DIST1,Hainaut,mfs


### Adding distributor actors to SND "belgium" circus

In [6]:
distributors = pd.read_csv("source_data/agent/distributor.csv").drop("agent_class", axis=1)
distributors = distributors[distributors["distributor_type"] != "origin"].drop("distributor_type", axis=1)

l1_names = ["DIST1", "DIST2", "DIST3", "DIST4"]
dist_l1 = distributors[distributors["agent_id"].isin(l1_names)] 
dist_l2 = distributors[~distributors["agent_id"].isin(l1_names)] 

dist_l1.head()

,agent_id,agent_name,agent_contact_name,agent_contact_phone
1,DIST1,HEINZ DISTRIB 6848,Cristopher Coles,32433162415
2,DIST2,AMORA DISTRIB 6189,Darron Craver,32497416595
3,DIST3,DEVOS & LEMMENS DISTRIB 3093,Cristopher Coles,32434046530
4,DIST4,DEVOS & LEMMENS DISTRIB 190,Adina Pacheo,32495097646


In [7]:
dist_l2.head(4)

,agent_id,agent_name,agent_contact_name,agent_contact_phone
5,DIST5,LA WILLIAMS DISTRIB 7331,Coreen Hoxie,32410717340
6,DIST6,LA WILLIAMS DISTRIB 4101,Danika Gierlach,32410526407
7,DIST7,CALVE DISTRIB 730,Scarlet Champine,32414549221
8,DIST8,AMORA DISTRIB 76,Adolfo Fuselier,32416587328


In [8]:
# basic distributor attributes
dist_l1_ac = circus.create_actor(name="dist_l1", ids=dist_l1.agent_id)
dist_l2_ac = circus.create_actor(name="dist_l2", ids=dist_l2.agent_id)

dist_l1_ac.create_attribute(name="NAME",          init_values=dist_l1.agent_name );
dist_l1_ac.create_attribute(name="CONTACT_NAME",  init_values=dist_l1.agent_contact_name );
dist_l1_ac.create_attribute(name="CONTACT_PHONE", init_values=dist_l1.agent_contact_phone );

dist_l2_ac.create_attribute(name="NAME",          init_values=dist_l2.agent_name );
dist_l2_ac.create_attribute(name="CONTACT_NAME",  init_values=dist_l2.agent_contact_name );
dist_l2_ac.create_attribute(name="CONTACT_PHONE", init_values=dist_l2.agent_contact_phone );


In [9]:
# relationship from distributor l1 to distributor l2, for each product
rels_grp = dealer_l2_to_l1_rel.groupby("product_type_id")

for product in dealer_l2_to_l1_rel.product_type_id.unique():
    rel_df = rels_grp.get_group(product)
    rel = dist_l2_ac.create_relationship(name="{}_l1_distributor".format(product))
    rel.add_relations(from_ids=rel_df.dist_l2, to_ids=rel_df.dist_l1)
    

# Sites

## Basic site attributes

In [10]:
cells = pd.read_csv("source_data/geography/cell.csv")
cells.site_name = cells.site_name.str.upper().str.replace(" ", "_")

# removing the 3 random sites 
cells = cells[~(cells.site_name.str[:11] == "SITE_RANDOM")]

sites = cells[["site_name", "site_longitude", "site_latitude", "geo_level1_id", "site_urban"]].drop_duplicates()

sites.shape

(4208, 5)

In [11]:
sites.sample(8)

,site_name,site_longitude,site_latitude,geo_level1_id,site_urban
2286,SITE_017824,4.535386,50.724627,LVL1_Rixensart,False
48306,SITE_011141,4.403788,50.874687,LVL1_Evere,True
16140,SITE_001995,4.184920,49.971313,LVL1_Momignies,False
30156,SITE_001041,4.705190,50.874425,LVL1_Louvain,True
21144,SITE_021670,4.948446,51.323123,LVL1_Turnhout,True
43908,SITE_002480,5.095653,50.296152,LVL1_Ciney,False
28800,SITE_007911,3.722614,51.055789,LVL1_Gand,True
34794,SITE_005562,4.423941,50.856170,LVL1_Woluwe-Saint-Lambert,True


## Linking sites to distributors

I'm adding here the link to the distibutor l1 and l2 for each product directly to the site. 
This greatly helps connecting the POS to the corresponding distributor in the circus

In [12]:
sites_gl3 = pd.merge(left=sites, right=geo_l1_to_l3, on="geo_level1_id")[["site_name", "geo_level3_id"]]

site_product_dists = pd.merge(left=sites_gl3, right=dealers_per_geo_l3, on="geo_level3_id")\
    .drop("geo_level3_id", axis=1)\
    .set_index(["site_name", "product"])\
    .stack().unstack(1).unstack()
    
site_product_dists.head()

product     electronic_recharge         handset             mfs          \
                        dist_l2 dist_l1 dist_l2 dist_l1 dist_l2 dist_l1   
site_name                                                                 
SITE_001002               DIST6   DIST2   DIST6   DIST2  DIST25   DIST1   
SITE_001003              DIST22   DIST2  DIST22   DIST2   DIST9   DIST1   
SITE_001004              DIST23   DIST3  DIST20   DIST4  DIST20   DIST4   
SITE_001005               DIST6   DIST2   DIST6   DIST2  DIST25   DIST1   
SITE_001006              DIST14   DIST2  DIST14   DIST2  DIST21   DIST1   

product     physical_recharge             sim          
                      dist_l2 dist_l1 dist_l2 dist_l1  
site_name                                              
SITE_001002             DIST6   DIST2  DIST25   DIST1  
SITE_001003            DIST22   DIST2   DIST9   DIST1  
SITE_001004            DIST23   DIST3  DIST23   DIST3  
SITE_001005             DIST6   DIST2  DIST25   DIST1  
SITE_001006            DIST14   DIST2  DIST21   DIST1

In [13]:
#
site_product_dists_flat = site_product_dists.copy()
site_product_dists_flat.columns =  ['__'.join(col).strip() for col in site_product_dists.columns.values]        
site_product_dists_flat = site_product_dists_flat.reset_index()

sites_with_dist = pd.merge(left=sites, right=site_product_dists_flat, on="site_name")
sites_with_dist.head()

,site_name,site_longitude,site_latitude,geo_level1_id,site_urban,electronic_recharge__dist_l2,electronic_recharge__dist_l1,handset__dist_l2,handset__dist_l1,mfs__dist_l2,mfs__dist_l1,physical_recharge__dist_l2,physical_recharge__dist_l1,sim__dist_l2,sim__dist_l1
0,SITE_001718,4.088464,50.742858,LVL1_Herne,True,DIST22,DIST2,DIST22,DIST2,DIST9,DIST1,DIST22,DIST2,DIST9,DIST1
1,SITE_021238,4.337534,50.835701,LVL1_Saint-Gilles,True,DIST7,DIST3,DIST12,DIST4,DIST12,DIST4,DIST7,DIST3,DIST7,DIST3
2,SITE_001869,4.434952,50.839264,LVL1_Woluwe-Saint-Lambert,True,DIST7,DIST3,DIST12,DIST4,DIST12,DIST4,DIST7,DIST3,DIST7,DIST3
3,SITE_004025,3.126656,50.817080,LVL1_Menin,False,DIST14,DIST2,DIST14,DIST2,DIST21,DIST1,DIST14,DIST2,DIST21,DIST1
4,SITE_004020,2.974190,51.207472,LVL1_Ostende,False,DIST14,DIST2,DIST14,DIST2,DIST21,DIST1,DIST14,DIST2,DIST21,DIST1


## Adding sites to the "belgium" SND circus

In [14]:
sites_actor = circus.create_actor(name="sites", ids=sites_with_dist.site_name)

sites_actor.create_attribute(name="LATITUDE",  init_values=sites_with_dist.site_latitude );
sites_actor.create_attribute(name="LONGITUDE",   init_values=sites_with_dist.site_longitude);
sites_actor.create_attribute(name="GEO_LEVEL_1", init_values=sites_with_dist.geo_level1_id );
sites_actor.create_attribute(name="URBAN", init_values=sites_with_dist.site_urban );

# include distributor of product as a circus actor attribute: 
for attr in site_product_dists_flat.columns: 
    sites_actor.create_attribute(name=attr,  init_values=sites_with_dist[attr]);

# adds a relationship from each site to all cells
cell_rel = sites_actor.create_relationship(name="CELLS")
cell_rel.add_relations(from_ids=cells.site_name,to_ids=cells.cell_id)

In [15]:
circus.actors["sites"].to_dataframe().sample(4)

,URBAN,sim__dist_l2,handset__dist_l1,site_name,sim__dist_l1,LONGITUDE,handset__dist_l2,mfs__dist_l1,mfs__dist_l2,LATITUDE,electronic_recharge__dist_l2,electronic_recharge__dist_l1,GEO_LEVEL_1,physical_recharge__dist_l1,physical_recharge__dist_l2
site_name,,,,,,,,,,,,,,,
SITE_006457,False,DIST21,DIST2,SITE_006457,DIST1,2.762690,DIST14,DIST1,DIST21,51.109297,DIST14,DIST2,LVL1_Nieuport,DIST2,DIST14
SITE_006520,False,DIST21,DIST2,SITE_006520,DIST1,2.750963,DIST14,DIST1,DIST21,51.129141,DIST14,DIST2,LVL1_Nieuport,DIST2,DIST14
SITE_003633,True,DIST15,DIST4,SITE_003633,DIST3,3.497074,DIST16,DIST4,DIST16,50.953472,DIST15,DIST3,LVL1_Zulte,DIST3,DIST15
SITE_003706,True,DIST25,DIST2,SITE_003706,DIST1,5.306211,DIST6,DIST1,DIST25,51.040835,DIST6,DIST2,LVL1_Heusden-Zolder,DIST2,DIST6


# Saving circus

In [16]:
circus.save_to_db(overwrite=True)